# Homework 1 (include all parts from challenge 1+optional, 2 and homework)
Authors:
- Nazarii Drushchak
- Igor Babin
- Uliana Zbezhkhovska

In [1]:
!pip install findspark
!pip install -q annoy
!pip install -q joblib
!pip install -q joblibspark

In [66]:
import findspark
findspark.init()

In [3]:
import pyspark
import numpy as np
from tqdm import tqdm

from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import MinHashLSH
from pyspark.sql.functions import col, avg, when
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover
from pyspark.ml.feature import Word2Vec
from annoy import AnnoyIndex
from sklearn.neighbors import NearestNeighbors
from pyspark.sql.functions import monotonically_increasing_id

from joblib import Parallel
from joblibspark import register_spark

In [4]:
sc = pyspark.SparkContext('local[*]')
spark = SparkSession(sc)
spark

## Challenge I

In [5]:
!wget http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2023-09-03/visualisations/listings.csv

--2023-11-09 06:34:52--  http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2023-09-03/visualisations/listings.csv
Resolving data.insideairbnb.com (data.insideairbnb.com)... 16.182.72.173, 52.217.72.19, 16.182.105.165, ...
Connecting to data.insideairbnb.com (data.insideairbnb.com)|16.182.72.173|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1698431 (1.6M) [application/csv]
Saving to: ‘listings.csv’

listings.csv        100%[===================>]   1.62M  1.79MB/s    in 0.9s    

2023-11-09 06:34:53 (1.79 MB/s) - ‘listings.csv’ saved [1698431/1698431]



In [6]:
df = spark.read.csv("listings.csv", header=True, multiLine=True)
df.show(5)

+------+--------------------+-------+---------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+
|    id|                name|host_id|host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|             license|
+------+--------------------+-------+---------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+
|761411|Condo in Amsterda...|4013546|   Xsjong|               NULL|   Noord-Oost|52.40164|  4.95106|   Private room|   61|             3|              303| 2023-08-19|  

Tokenize (remove punctuation and split by word), you can do it in pure python or using ml-lib tokenizer

In [7]:
tokenizer = Tokenizer(inputCol="name", outputCol="words")
wordData = tokenizer.transform(df)
wordData.show(5)

+------+--------------------+-------+---------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+--------------------+
|    id|                name|host_id|host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|             license|               words|
+------+--------------------+-------+---------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+--------------------+
|761411|Condo in Amsterda...|4013546|   Xsjong|               NULL|   Noord-Oost|52.40164|  4.95106|   Pri

Remove stopwords using ML-LIB stopwordsremover, and store in a new column called “CleanTokens”

In [8]:
remover = StopWordsRemover(inputCol="words", outputCol="CleanTokens")
cleanData = remover.transform(wordData)
cleanData.show(5)

+------+--------------------+-------+---------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+--------------------+--------------------+
|    id|                name|host_id|host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|             license|               words|         CleanTokens|
+------+--------------------+-------+---------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+--------------------+--------------------+
|761411|Condo in Amsterda...|4013546|   Xsj

But we don’t have a stopwordsremover for all language and contexts. Create your own list of stopwords from this text (think: what is a stopword?) Remove stopwords again, and store in column “MyCleanTokens”

In [9]:
stop_list = ['the', 'a', 'an', 'another', "for", "an", "nor", "but", "or", "yet", "so", 
                                      "in", "under", "towards", "before"]
remover = StopWordsRemover(stopWords=stop_list, inputCol='words', outputCol='MyCleanTokens')
cleanData = remover.transform(cleanData)
cleanData.show(5)

+------+--------------------+-------+---------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+--------------------+--------------------+--------------------+
|    id|                name|host_id|host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|             license|               words|         CleanTokens|       MyCleanTokens|
+------+--------------------+-------+---------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+--------------------+--------------------+--

Perform TFIDF in a new column called “VectorSpace”

In [10]:
hashingTF = HashingTF(inputCol="MyCleanTokens", outputCol="VectorSpace", numFeatures=20)
featurizedData = hashingTF.transform(cleanData)

idf = IDF(inputCol="VectorSpace", outputCol="features")
idfModel = idf.fit(featurizedData)
results = idfModel.transform(featurizedData)

results.select("MyCleanTokens", "features").show(5)

+--------------------+--------------------+
|       MyCleanTokens|            features|
+--------------------+--------------------+
|[condo, amsterdam...|(20,[1,2,7,10,11,...|
|[rental, unit, am...|(20,[1,2,5,11,12,...|
|[boat, amsterdam,...|(20,[0,1,2,5,11,1...|
|[houseboat, amste...|(20,[0,1,5,6,11,1...|
|[rental, unit, am...|(20,[1,2,9,11,12,...|
+--------------------+--------------------+
only showing top 5 rows



## Homework (Optional)

In a new column(‘word2vec’), repeat the procedure using word2vec instead of TF-IDF.

In [11]:
word2Vec = Word2Vec(vectorSize=20, minCount=0, inputCol="MyCleanTokens", outputCol="word2vec")
model = word2Vec.fit(results)
result = model.transform(results)

result.select("MyCleanTokens", "word2vec").show(5)

+--------------------+--------------------+
|       MyCleanTokens|            word2vec|
+--------------------+--------------------+
|[condo, amsterdam...|[0.12229608697816...|
|[rental, unit, am...|[0.15610642857583...|
|[boat, amsterdam,...|[-0.0039486894384...|
|[houseboat, amste...|[-0.1444090648482...|
|[rental, unit, am...|[0.22863571302344...|
+--------------------+--------------------+
only showing top 5 rows



Show first row word2vec vector

In [12]:
result.select("word2vec").first()

Row(word2vec=DenseVector([0.1223, 0.2137, 0.2644, -0.0106, 0.4746, 0.1475, -0.0027, 0.4228, 0.0387, -0.115, 0.3691, -0.1899, -0.1823, 0.278, -0.0575, -0.332, 0.2215, -0.4446, 0.3223, 0.1145]))

## Challenge II

Take the first 500 flats in the list

In [13]:
mysample = result.limit(500)
mysample.count()

500

Find the 3 nearest neighbors for each element in that subset (candidates and query points are within the sample of 500) USING KNN

In [14]:
mysample_pd = mysample.toPandas()
tfidf = mysample_pd['features'].tolist()
text = mysample_pd['name'].tolist()
id_ = mysample_pd['id'].tolist()

# fit nearest neighbors
nbrs = NearestNeighbors(n_neighbors=4).fit(tfidf)
distances, indices = nbrs.kneighbors(tfidf[:5])

# show 3 nearest neighbors for first row except itself
print('id', [id_[i] for i in indices[0]])

id ['761411', '634170', '721291', '730916']


Find the 3 nearest neighbors for each element in that subset (candidates and query points are within the sample of 500) USING LSH with sklearn

In [15]:
# IT IS DEPRECATED or Install 3 years old version of sklearn 0.16.1
try:
    from sklearn.neighbors import LSHForest
    
    mysample_pd = mysample.toPandas()
    tfidf = mysample_pd['features'].tolist()
    text = mysample_pd['name'].tolist()
    id_ = mysample_pd['id'].tolist()
    
    lshf = LSHForest(random_state=42)
    lshf.fit(tfidf)
    
    # get the feature vectore of the first row
    query = tfidf[0]
    id_ = id_[0]
    
    # show 3 nearest neighbors for first row except itself
    distances, indices = lshf.kneighbors([query], n_neighbors=4)
    for i in range(1, len(distances[0])):
        print("distance: ", distances[0][i], "id: ", id_[indices[0][i]])
except ImportError:
    print("LSHForest could not be imported")

LSHForest could not be imported


Find the 3 nearest neighbors for each element in that subset (candidates and query points are within the sample of 500) USING LSH with pyspark

In [16]:
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=3)
model = mh.fit(mysample)

# get the feature vector of the first row
key =  mysample.select("features").take(1)[0].features
id_ = mysample.select("id").take(1)[0].id


# show 3 nearest neighbors for first row except itself
model.approxNearestNeighbors(mysample, key, 4).filter(col("id") != id_).show()

+-------+--------------------+-------+----------------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|     id|                name|host_id|       host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|             license|               words|         CleanTokens|       MyCleanTokens|         VectorSpace|            features|            word2vec|              hashes|           distCol|
+-------+--------------------+-------+----------------+-------------------+-------------+--------+

## Challenge III: Homework

![image.png](HW1_task.jpeg)

## Load Barcelona Airbnb data

In [17]:
!wget http://data.insideairbnb.com/spain/catalonia/barcelona/2023-09-06/visualisations/listings.csv -O listings_barcelona.csv 

--2023-11-09 06:35:03--  http://data.insideairbnb.com/spain/catalonia/barcelona/2023-09-06/visualisations/listings.csv
Resolving data.insideairbnb.com (data.insideairbnb.com)... 16.182.72.173, 52.217.72.19, 16.182.105.165, ...
Connecting to data.insideairbnb.com (data.insideairbnb.com)|16.182.72.173|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3664972 (3.5M) [application/csv]
Saving to: ‘listings_barcelona.csv’

listings_barcelona. 100%[===================>]   3.50M  2.25MB/s    in 1.6s    

2023-11-09 06:35:05 (2.25 MB/s) - ‘listings_barcelona.csv’ saved [3664972/3664972]



In [18]:
df = spark.read.csv("listings_barcelona.csv", header=True, multiLine=True)
df.show(5)

+------+--------------------+-------+----------------+-------------------+--------------------+-----------------+-----------------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-----------+
|    id|                name|host_id|       host_name|neighbourhood_group|       neighbourhood|         latitude|        longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|    license|
+------+--------------------+-------+----------------+-------------------+--------------------+-----------------+-----------------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-----------+
| 18674|Rental unit in Ba...|  71615|Mireia And Maria|           Eixample|  la Sagrada Família|        

In [19]:
df.count()

18086

## Load wikipedia data

Load file from: https://pageviews.wmcloud.org/topviews/?project=uk.wikipedia.org&platform=all-access&date=2023-09&excludes= in CSV format

In [20]:
df = spark.read.csv("work/topviews-2023_09.csv", header=True, multiLine=True)
df.show(5)

+--------------------+-----+-------+------+
|                Page|Edits|Editors| Views|
+--------------------+-----+-------+------+
|Умєров Рустем Енв...|   54|     34|127352|
|             Ukr.net|    2|      1| 97183|
|             Україна|    8|      6| 94568|
|Кадиров Рамзан Ах...|   15|      8| 86347|
|    Нагірний Карабах|   32|     12| 79264|
+--------------------+-----+-------+------+
only showing top 5 rows



In [21]:
df.count()

991

## Additional function

In [78]:
use_stopwords = True  # Use True or False
use_custom_stopwords = False  # Use True or False
latent_features = 20  # Dimension of features
nearest = 3  # Number of nearest neighbors

# # Register Spark to be used by joblib
register_spark()

def timeit(func):
    def timed(*args, **kwargs):
        import time
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print("Time elapsed for " + func.__name__ + ": " + str(end - start))
        return result

    return timed

def read_data(spark, data):
    if data == "barcelona":
        df = spark.read.csv("listings_barcelona.csv", header=True, multiLine=True)
    elif data == "titles":
        df = spark.read.csv("work/topviews-2023_09.csv", header=True, multiLine=True)
        # add id column as row number as string
        df = df.withColumn("id", monotonically_increasing_id().cast("string"))
    else:
        raise ValueError("Invalid data")
    return df

def limit_data(df, limit=50): # -1 means no limit
    if limit > 0:
        df = df.limit(limit)
    return df

def get_features(df, input_col="name", output_col="features", type_features="tfidf"):
    tokenizer = Tokenizer(inputCol=input_col, outputCol="words")
    df = tokenizer.transform(df)

    if use_stopwords:
        if use_custom_stopwords:
            remover = StopWordsRemover(stopWords=stop_list, inputCol="words", outputCol="clean_tokens")
        else:
            remover = StopWordsRemover(inputCol="words", outputCol="clean_tokens")
        df = remover.transform(df)
        df = df.drop("words")
        df = df.withColumnRenamed("clean_tokens", "words")

    if type_features == "tfidf":
        hashing = HashingTF(inputCol="words", outputCol="hash", numFeatures=latent_features)
        df = hashing.transform(df)

        idf = IDF(inputCol="hash", outputCol=output_col)
        model = idf.fit(df)
        df = model.transform(df)
    elif type_features == "word_to_vec":
        word_vec = Word2Vec(vectorSize=latent_features, minCount=0, inputCol="words", outputCol=output_col)
        model = word_vec.fit(df)
        df = model.transform(df)
    else:
        raise ValueError("Invalid feature " + type_features)

    return df

@timeit
def compute_gt(ds, spark, k=nearest, input_col="features", output_col="gt_neighbors"):
    df = ds.toPandas()
    features = df[input_col].tolist()
    model = NearestNeighbors(n_neighbors=k + 1, algorithm='ball_tree').fit(features)
    _, indices = model.kneighbors(features)
    # remove self from neighbors
    indices = [Vectors.dense(df["id"][np.delete(ind, np.where(ind == i))].values) for i, ind in enumerate(indices)]
    df[output_col] = indices

    return spark.createDataFrame(df)

@timeit
def lsh_prediction(ds, spark, k=nearest, input_col="features",
                   output_col="ann_neighbors", num_hash_tables=3):
    model = MinHashLSH(inputCol=input_col, outputCol=output_col, numHashTables=num_hash_tables)
    model = model.fit(ds)

    # TODO There should be something better than this
    pred = []
    for i in tqdm(ds.collect()):
        id_ = i["id"]
        key = i[input_col]
        pred.append(model.approxNearestNeighbors(ds, key, k + 1).filter(col("id") != id_).select("id").collect())

    pred = [Vectors.dense([i["id"] for i in ann]) for ann in pred]
    df = ds.toPandas()
    df[output_col] = pred
    ds = spark.createDataFrame(df)

    return ds

@timeit
def grid_search_lsh(ds, spark, k=nearest, input_col="features", output_col="ann_neighbors"):
    param_grid = [5, 10, 20, 100]
    results = []
    print("="*50)
    print("Grid search for LSH")

    # Run the grid search in parallel
    with Parallel(n_jobs=-1, backend="spark") as parallel:
        for num_hash_table in param_grid:
            ds_ = lsh_prediction(ds, spark, k=k, input_col=input_col, output_col=output_col,
                                num_hash_tables=num_hash_table)
            acc = evaluation(ds_)
            print(f"Method: LSH - Num Hash Tables: {num_hash_table} - Accuracy: {acc}\n")
            results.append((num_hash_table, acc))
    print("="*50, "\n\n")
    return results

@timeit
def annoy_prediction(ds, k=nearest, input_col="features", output_col="ann_neighbors", metric='angular', tree=10):
    df = ds.toPandas()
    features = df[input_col].tolist()
    f = len(features[0])
    t = AnnoyIndex(f, metric=metric)
    for i, v in enumerate(features):
        t.add_item(i, v)
    t.build(tree)
    pred = []
    for i in tqdm(features):
        pred.append(t.get_nns_by_vector(i, k + 1, include_distances=False)[1:])
    pred = [Vectors.dense([df["id"][i] for i in ann]) for ann in pred]
    df[output_col] = pred
    ds = spark.createDataFrame(df)

    return ds

@timeit
def grid_search_annoy(ds, k=nearest, input_col="features", output_col="ann_neighbors"):
    metrics = ['angular', 'euclidean', 'dot']
    trees = [10, 100, 1000]
    param_grid = [(metric, tree) for metric in metrics for tree in trees]
    results = []
    print("="*50)
    print("Grid search for Annoy")

    # Run the grid search in parallel
    with Parallel(n_jobs=-1, backend="spark") as parallel:
        for metric, tree in param_grid:
            ds = annoy_prediction(ds, k=k, input_col=input_col, output_col=output_col, metric=metric, tree=tree)
            acc = evaluation(ds)
            print(f"Method: Annoy - Metric: {metric} - Tree: {tree} - Accuracy: {acc}\n")
            results.append((metric, tree, acc))
    print("="*50, "\n\n")
    return results

def evaluation(ds):
    acc = 0
    for i in ds.collect():
        gt = i["gt_neighbors"]
        ann = i["ann_neighbors"]
        gt.sort(), ann.sort()
        acc += len(set(gt).intersection(set(ann)))
    acc /= len(ds.collect()) * nearest

    return acc

## Barcelona dataset
### type_features: `Tf-idf`
**Try LSH method from Pyspark(not optimized) and LSH method from Annoy**

In [70]:
limit = -1  # Use -1 for no limit
data = "barcelona"  # Use "barcelona" or "titles"
nearest = 3
use_stopwords = True  # Use True or False
use_custom_stopwords = False  # Use True or False
latent_features = 20  # Dimension of features
type_features = "tfidf"  # Use "tfidf" or "word_to_vec"

In [24]:
print("Reading data for " + data + " with limit " + str(limit) + " and features " + type_features + " ...\n")

df = read_data(spark, data)
df = limit_data(df, limit)

df = get_features(df, type_features=type_features)

print("Calculating gt neighbors nearest neighbors, could take a while...")
df = compute_gt(df, spark)

grid_search_lsh(df, spark)

Reading data for barcelona with limit -1 and features tfidf ...

Calculating gt neighbors nearest neighbors, could take a while...
Time elapsed for compute_gt: 12.146341562271118
Grid search for LSH


100%|██████████| 18086/18086 [1:10:02<00:00,  4.30it/s]


Time elapsed for lsh_prediction: 4207.41507434845
Method: LSH - Num Hash Tables: 5 - Accuracy: 0.43508791330310737



100%|██████████| 18086/18086 [1:13:11<00:00,  4.12it/s]


Time elapsed for lsh_prediction: 4396.357168674469
Method: LSH - Num Hash Tables: 10 - Accuracy: 0.4507353754285082



100%|██████████| 18086/18086 [1:22:12<00:00,  3.67it/s]


Time elapsed for lsh_prediction: 4937.006164073944
Method: LSH - Num Hash Tables: 20 - Accuracy: 0.45020089203435437



100%|██████████| 18086/18086 [2:38:07<00:00,  1.91it/s]  


Time elapsed for lsh_prediction: 9491.71913075447
Method: LSH - Num Hash Tables: 100 - Accuracy: 0.45020089203435437



Time elapsed for grid_search_lsh: 23044.375668525696


[(5, 0.43508791330310737),
 (10, 0.4507353754285082),
 (20, 0.45020089203435437),
 (100, 0.45020089203435437)]

In [25]:
print("Reading data for " + data + " with limit " + str(limit) + " and features " + type_features + " ...\n")

df = read_data(spark, data)
df = limit_data(df, limit)

df = get_features(df, type_features=type_features)

print("Calculating gt neighbors nearest neighbors, could take a while...")
df = compute_gt(df, spark)

grid_search_annoy(df, k=nearest, input_col="features", output_col="ann_neighbors")

Reading data for barcelona with limit -1 and features tfidf ...

Calculating gt neighbors nearest neighbors, could take a while...
Time elapsed for compute_gt: 12.171573162078857
Grid search for Annoy


100%|██████████| 18086/18086 [00:01<00:00, 12477.53it/s]


Time elapsed for annoy_prediction: 6.527449369430542
Method: Annoy - Metric: angular - Tree: 10 - Accuracy: 0.2487743742858196



100%|██████████| 18086/18086 [00:03<00:00, 5752.82it/s]


Time elapsed for annoy_prediction: 8.726189613342285
Method: Annoy - Metric: angular - Tree: 100 - Accuracy: 0.657967488665266



...

100%|██████████| 18086/18086 [00:03<00:00, 5468.35it/s]


Time elapsed for annoy_prediction: 8.755682229995728
Method: Annoy - Metric: dot - Tree: 100 - Accuracy: 0.028898964208043054



100%|██████████| 18086/18086 [00:25<00:00, 720.14it/s]


Time elapsed for annoy_prediction: 36.909693002700806
Method: Annoy - Metric: dot - Tree: 1000 - Accuracy: 0.016734859375575954



Time elapsed for grid_search_annoy: 181.61139702796936


[('angular', 10, 0.2487743742858196),
 ('angular', 100, 0.657967488665266),
 ('angular', 1000, 0.9122525710494305),
 ('euclidean', 10, 0.49808323196579307),
 ('euclidean', 100, 0.7797928416086107),
 ('euclidean', 1000, 0.9194588816395739),
 ('dot', 10, 0.016274097828891592),
 ('dot', 100, 0.028898964208043054),
 ('dot', 1000, 0.016734859375575954)]

## Barcelona dataset
### type_features: `Word2vec`
**Try LSH method from Pyspark(not optimized) and LSH method from Annoy**

In [47]:
limit = -1  # Use -1 for no limit
data = "barcelona"  # Use "barcelona" or "titles"
nearest = 3
use_stopwords = True  # Use True or False
use_custom_stopwords = False  # Use True or False
latent_features = 20  # Dimension of features
type_features = "word_to_vec"  # Use "tfidf" or "word_to_vec"

In [27]:
print("Reading data for " + data + " with limit " + str(limit) + " and features " + type_features + " ...\n")

df = read_data(spark, data)
df = limit_data(df, limit)

df = get_features(df, type_features=type_features)

print("Calculating gt neighbors nearest neighbors, could take a while...")
df = compute_gt(df, spark)

grid_search_lsh(df, spark)

Reading data for barcelona with limit -1 and features word_to_vec ...

Calculating gt neighbors nearest neighbors, could take a while...
Time elapsed for compute_gt: 5.767719984054565
Grid search for LSH


100%|██████████| 18086/18086 [1:10:09<00:00,  4.30it/s]


Time elapsed for lsh_prediction: 4212.666715621948
Method: LSH - Num Hash Tables: 5 - Accuracy: 7.372184746949758e-05



100%|██████████| 18086/18086 [1:23:23<00:00,  3.61it/s]


Time elapsed for lsh_prediction: 5007.108249425888
Method: LSH - Num Hash Tables: 10 - Accuracy: 7.372184746949758e-05



100%|██████████| 18086/18086 [1:34:53<00:00,  3.18it/s]


Time elapsed for lsh_prediction: 5696.771778583527
Method: LSH - Num Hash Tables: 20 - Accuracy: 7.372184746949758e-05



100%|██████████| 18086/18086 [3:44:11<00:00,  1.34it/s]  


Time elapsed for lsh_prediction: 13454.690614700317
Method: LSH - Num Hash Tables: 100 - Accuracy: 7.372184746949758e-05



Time elapsed for grid_search_lsh: 28378.0657453537


[(5, 7.372184746949758e-05),
 (10, 7.372184746949758e-05),
 (20, 7.372184746949758e-05),
 (100, 7.372184746949758e-05)]

In [28]:
print("Reading data for " + data + " with limit " + str(limit) + " and features " + type_features + " ...\n")

df = read_data(spark, data)
df = limit_data(df, limit)

df = get_features(df, type_features=type_features)

print("Calculating gt neighbors nearest neighbors, could take a while...")
df = compute_gt(df, spark)

grid_search_annoy(df, k=nearest, input_col="features", output_col="ann_neighbors")

Reading data for barcelona with limit -1 and features word_to_vec ...

Calculating gt neighbors nearest neighbors, could take a while...
Time elapsed for compute_gt: 5.637991189956665
Grid search for Annoy


100%|██████████| 18086/18086 [00:00<00:00, 68194.55it/s]


Time elapsed for annoy_prediction: 2.9605634212493896
Method: Annoy - Metric: angular - Tree: 10 - Accuracy: 0.5126248663791515



100%|██████████| 18086/18086 [00:02<00:00, 8610.88it/s]


Time elapsed for annoy_prediction: 5.4291160106658936
Method: Annoy - Metric: angular - Tree: 100 - Accuracy: 0.6097718308820819



...

100%|██████████| 18086/18086 [00:01<00:00, 13459.48it/s]


Time elapsed for annoy_prediction: 4.712590217590332
Method: Annoy - Metric: dot - Tree: 100 - Accuracy: 0.0058055954882229345



100%|██████████| 18086/18086 [00:17<00:00, 1035.88it/s]


Time elapsed for annoy_prediction: 27.277600049972534
Method: Annoy - Metric: dot - Tree: 1000 - Accuracy: 0.0029120129750451547



Time elapsed for grid_search_annoy: 134.33812046051025


[('angular', 10, 0.5126248663791515),
 ('angular', 100, 0.6097718308820819),
 ('angular', 1000, 0.621346160934793),
 ('euclidean', 10, 0.5535957831103248),
 ('euclidean', 100, 0.6293081204614988),
 ('euclidean', 1000, 0.6516642707066239),
 ('dot', 10, 0.001585019720594198),
 ('dot', 100, 0.0058055954882229345),
 ('dot', 1000, 0.0029120129750451547)]

## Wikipedia dataset
### type_features: `Tf-idf`
**Try LSH method from Pyspark(not optimized) and LSH method from Annoy**

In [79]:
import pyspark.sql.functions as F
from pyspark.sql.types import T

In [80]:
limit = -1  # Use -1 for no limit
data = "titles"  # Use "barcelona" or "titles"
nearest = 3
use_stopwords = True  # Use True or False
use_custom_stopwords = False  # Use True or False
latent_features = 100  # Dimension of features
type_features = "tfidf"  # Use "tfidf" or "word_to_vec"

In [81]:
print("Reading data for " + data + " with limit " + str(limit) + " and features " + type_features + " ...\n")

df = read_data(spark, data)
df = limit_data(df, limit)

df = get_features(df, input_col="Page", type_features=type_features)
df = df.filter(~F.col('features').cast('string').contains('[]'))

print("Calculating gt neighbors nearest neighbors, could take a while...")
df = compute_gt(df, spark)

grid_search_lsh(df, spark)

Reading data for titles with limit -1 and features tfidf ...

Calculating gt neighbors nearest neighbors, could take a while...
Time elapsed for compute_gt: 1.0458683967590332
Grid search for LSH


100%|██████████| 990/990 [02:23<00:00,  6.89it/s]


Time elapsed for lsh_prediction: 144.47205090522766
Method: LSH - Num Hash Tables: 5 - Accuracy: 0.5932659932659933



100%|██████████| 990/990 [02:38<00:00,  6.24it/s]


Time elapsed for lsh_prediction: 159.42340731620789
Method: LSH - Num Hash Tables: 10 - Accuracy: 0.5959595959595959



100%|██████████| 990/990 [02:02<00:00,  8.06it/s]


Time elapsed for lsh_prediction: 123.37339949607849
Method: LSH - Num Hash Tables: 20 - Accuracy: 0.5969696969696969



100%|██████████| 990/990 [02:21<00:00,  6.97it/s]


Time elapsed for lsh_prediction: 142.37637209892273
Method: LSH - Num Hash Tables: 100 - Accuracy: 0.5973063973063973



Time elapsed for grid_search_lsh: 570.9063160419464


[(5, 0.5932659932659933),
 (10, 0.5959595959595959),
 (20, 0.5969696969696969),
 (100, 0.5973063973063973)]

In [82]:
print("Reading data for " + data + " with limit " + str(limit) + " and features " + type_features + " ...\n")

df = read_data(spark, data)
df = limit_data(df, limit)

df = get_features(df, input_col="Page", type_features=type_features)
df = df.filter(~F.col('features').cast('string').contains('[]'))

print("Calculating gt neighbors nearest neighbors, could take a while...")
df = compute_gt(df, spark)

grid_search_annoy(df, k=nearest, input_col="features", output_col="ann_neighbors")

Reading data for titles with limit -1 and features tfidf ...

Calculating gt neighbors nearest neighbors, could take a while...
Time elapsed for compute_gt: 0.8998205661773682
Grid search for Annoy


100%|██████████| 990/990 [00:00<00:00, 3442.67it/s]


Time elapsed for annoy_prediction: 0.7412104606628418
Method: Annoy - Metric: angular - Tree: 10 - Accuracy: 0.5616161616161616



100%|██████████| 990/990 [00:00<00:00, 3317.65it/s]


Time elapsed for annoy_prediction: 0.9903769493103027
Method: Annoy - Metric: angular - Tree: 100 - Accuracy: 0.807070707070707



...

100%|██████████| 990/990 [00:00<00:00, 3328.06it/s]


Time elapsed for annoy_prediction: 0.7676877975463867
Method: Annoy - Metric: dot - Tree: 100 - Accuracy: 0.17777777777777778



100%|██████████| 990/990 [00:00<00:00, 1567.40it/s]


Time elapsed for annoy_prediction: 1.5372910499572754
Method: Annoy - Metric: dot - Tree: 1000 - Accuracy: 0.1750841750841751



Time elapsed for grid_search_annoy: 11.711737632751465


[('angular', 10, 0.5616161616161616),
 ('angular', 100, 0.807070707070707),
 ('angular', 1000, 0.8245791245791246),
 ('euclidean', 10, 0.6047138047138048),
 ('euclidean', 100, 0.877104377104377),
 ('euclidean', 1000, 0.9097643097643098),
 ('dot', 10, 0.19932659932659932),
 ('dot', 100, 0.17777777777777778),
 ('dot', 1000, 0.1750841750841751)]

## Wikipedia dataset
### type_features: `Word2Vec`
**Try LSH method from Pyspark(not optimized) and LSH method from Annoy**

In [83]:
limit = -1  # Use -1 for no limit
data = "titles"  # Use "barcelona" or "titles"
nearest = 3
use_stopwords = True  # Use True or False
use_custom_stopwords = False  # Use True or False
latent_features = 20  # Dimension of features
type_features = "word_to_vec"  # Use "tfidf" or "word_to_vec"

In [84]:
print("Reading data for " + data + " with limit " + str(limit) + " and features " + type_features + " ...\n")

df = read_data(spark, data)
df = limit_data(df, limit)

df = get_features(df, input_col="Page", type_features=type_features)
df = df.filter(~F.col('features').cast('string').contains('[]'))

print("Calculating gt neighbors nearest neighbors, could take a while...")
df = compute_gt(df, spark)

grid_search_lsh(df, spark)

Reading data for titles with limit -1 and features word_to_vec ...

Calculating gt neighbors nearest neighbors, could take a while...
Time elapsed for compute_gt: 0.3333303928375244
Grid search for LSH


100%|██████████| 990/990 [02:11<00:00,  7.55it/s]


Time elapsed for lsh_prediction: 131.77543473243713
Method: LSH - Num Hash Tables: 5 - Accuracy: 0.0030303030303030303



100%|██████████| 990/990 [02:27<00:00,  6.72it/s]


Time elapsed for lsh_prediction: 147.77966213226318
Method: LSH - Num Hash Tables: 10 - Accuracy: 0.0030303030303030303



100%|██████████| 990/990 [02:23<00:00,  6.89it/s]


Time elapsed for lsh_prediction: 144.15844702720642
Method: LSH - Num Hash Tables: 20 - Accuracy: 0.0030303030303030303



100%|██████████| 990/990 [02:32<00:00,  6.49it/s]


Time elapsed for lsh_prediction: 152.8714427947998
Method: LSH - Num Hash Tables: 100 - Accuracy: 0.0030303030303030303



Time elapsed for grid_search_lsh: 578.0813789367676


[(5, 0.0030303030303030303),
 (10, 0.0030303030303030303),
 (20, 0.0030303030303030303),
 (100, 0.0030303030303030303)]

In [85]:
print("Reading data for " + data + " with limit " + str(limit) + " and features " + type_features + " ...\n")

df = read_data(spark, data)
df = limit_data(df, limit)

df = get_features(df, input_col="Page", type_features=type_features)
df = df.filter(~F.col('features').cast('string').contains('[]'))

print("Calculating gt neighbors nearest neighbors, could take a while...")
df = compute_gt(df, spark)

grid_search_annoy(df, k=nearest, input_col="features", output_col="ann_neighbors")

Reading data for titles with limit -1 and features word_to_vec ...

Calculating gt neighbors nearest neighbors, could take a while...
Time elapsed for compute_gt: 0.30584263801574707
Grid search for Annoy


100%|██████████| 990/990 [00:00<00:00, 98369.21it/s]


Time elapsed for annoy_prediction: 0.3739008903503418
Method: Annoy - Metric: angular - Tree: 10 - Accuracy: 0.40606060606060607



100%|██████████| 990/990 [00:00<00:00, 14765.00it/s]


Time elapsed for annoy_prediction: 0.3154134750366211
Method: Annoy - Metric: angular - Tree: 100 - Accuracy: 0.5404040404040404



...

100%|██████████| 990/990 [00:00<00:00, 15471.25it/s]


Time elapsed for annoy_prediction: 0.2469027042388916
Method: Annoy - Metric: dot - Tree: 100 - Accuracy: 0.05858585858585859



100%|██████████| 990/990 [00:00<00:00, 1358.73it/s]


Time elapsed for annoy_prediction: 1.3285729885101318
Method: Annoy - Metric: dot - Tree: 1000 - Accuracy: 0.05387205387205387



Time elapsed for grid_search_annoy: 8.422160387039185


[('angular', 10, 0.40606060606060607),
 ('angular', 100, 0.5404040404040404),
 ('angular', 1000, 0.5414141414141415),
 ('euclidean', 10, 0.5276094276094276),
 ('euclidean', 100, 0.9693602693602693),
 ('euclidean', 1000, 1.0),
 ('dot', 10, 0.0861952861952862),
 ('dot', 100, 0.05858585858585859),
 ('dot', 1000, 0.05387205387205387)]

# Results and conclusions:

Your report should include:
- The accuracy of your model with at least 4 combinations of parameters
- The computation time spent in the parameter tuning procedure (specify also the characteristics of the machine(s) that you have used)
- What happen if we tune the parameters using Airbnb and the run with those parameters in The Wikipedia Dataset? What is the difference in accuracy with the parameters tuned directly in Wikipedia data?

1. We run GridSearch for 4 combinations of parameters for each dataset. The results are shown in the table below:

| Dataset |Model Type|Feature Type |Parameters | Accuracy | Time |
| --- | --- | --- | --- | --- | --- |
| Barcelona | MinHashLSH | Tf-idf | numHashTables=5 | 0.435 | 4207.415 |
| Barcelona | MinHashLSH | Tf-idf | numHashTables=10 | 0.450 | 4396.357 |
| Barcelona | MinHashLSH | Tf-idf | numHashTables=20 | 0.450 | 4937.006 |
| Barcelona | MinHashLSH | Tf-idf | numHashTables=100 | 0.450 | 9491.719 |
| Barcelona | Annoy | Tf-idf | metric=angular, n_trees=10 | 0.248 | 6.527 |
| Barcelona | Annoy | Tf-idf | metric=angular, n_trees=100 | 0.657 | 8.726 |
| Barcelona | Annoy | Tf-idf | metric=angular, n_trees=1000 | 0.912 | 38.112 |
| Barcelona | Annoy | Tf-idf | metric=euclidean, n_trees=10 | 0.498 | 6.430 |
| Barcelona | Annoy | Tf-idf | metric=euclidean, n_trees=100 | 0.779 | 8.464 |
| `Barcelona` | `Annoy` | `Tf-idf` | `metric=euclidean, n_trees=1000` | `0.919` | `33.919` |
| Barcelona | Annoy | Tf-idf | metric=dot, n_trees=10 | 0.016 | 6.211 |
| Barcelona | Annoy | Tf-idf | metric=dot, n_trees=100 | 0.028 | 8.755 |
| Barcelona | Annoy | Tf-idf | metric=dot, n_trees=1000 | 0.016 | 36.909 |
| Barcelona | MinHashLSH | Word2Vec | numHashTables=5 | 0.000 | 4212.667 |
| Barcelona | MinHashLSH | Word2Vec | numHashTables=10 | 0.000 | 5007.108 |
| Barcelona | MinHashLSH | Word2Vec | numHashTables=20 | 0.000 | 5696.772 |
| Barcelona | MinHashLSH | Word2Vec | numHashTables=100 | 0.000 | 13454.691 |
| Barcelona | Annoy | Word2Vec | metric=angular, n_trees=10 | 0.512 | 2.961 |
| Barcelona | Annoy | Word2Vec | metric=angular, n_trees=100 | 0.610 | 5.429 |
| Barcelona | Annoy | Word2Vec | metric=angular, n_trees=1000 | 0.621 | 35.244 |
| Barcelona | Annoy | Word2Vec | metric=euclidean, n_trees=10 | 0.554 | 2.932 |
| Barcelona | Annoy | Word2Vec | metric=euclidean, n_trees=100 | 0.629 | 4.919 |
| Barcelona | Annoy | Word2Vec | metric=euclidean, n_trees=1000 | 0.652 | 31.751 |
| Barcelona | Annoy | Word2Vec | metric=dot, n_trees=10 | 0.002 | 2.997 |
| Barcelona | Annoy | Word2Vec | metric=dot, n_trees=100 | 0.006 | 4.713 |
| Barcelona | Annoy | Word2Vec | metric=dot, n_trees=1000 | 0.003 | 27.278 |
| Wikipedia | MinHashLSH | Tf-idf | numHashTables=5 | 0.593 | 144.472 |
| Wikipedia | MinHashLSH | Tf-idf | numHashTables=10 | 0.596 | 159.423 |
| Wikipedia | MinHashLSH | Tf-idf | numHashTables=20 | 0.596 | 123.373 |
| Wikipedia | MinHashLSH | Tf-idf | numHashTables=100 | 0.597 | 142.376 |
| Wikipedia | Annoy | Tf-idf | metric=angular, n_trees=10 | 0.561 | 0.741 |
| Wikipedia | Annoy | Tf-idf | metric=angular, n_trees=100 | 0.807 | 0.990 |
| Wikipedia | Annoy | Tf-idf | metric=angular, n_trees=1000 | 0.825 | 1.409 |
| Wikipedia | Annoy | Tf-idf | metric=euclidean, n_trees=10 | 0.605 | 0.778 |
| Wikipedia | Annoy | Tf-idf | metric=euclidean, n_trees=100 | 0.877 | 0.814 |
| Wikipedia | Annoy | Tf-idf | metric=euclidean, n_trees=1000 | 0.910 | 1.304 |
| Wikipedia | Annoy | Tf-idf | metric=dot, n_trees=10 | 0.199 | 0.823 |
| Wikipedia | Annoy | Tf-idf | metric=dot, n_trees=100 | 0.178 | 0.768 |
| Wikipedia | Annoy | Tf-idf | metric=dot, n_trees=1000 | 0.175 | 1.537 |
| Wikipedia | MinHashLSH | Word2Vec | numHashTables=5 | 0.003 | 131.775 |
| Wikipedia | MinHashLSH | Word2Vec | numHashTables=10 | 0.003 | 147.779 |
| Wikipedia | MinHashLSH | Word2Vec | numHashTables=20 | 0.003 | 144.158 |
| Wikipedia | MinHashLSH | Word2Vec | numHashTables=100 | 0.003 | 152.871 |
| Wikipedia | Annoy | Word2Vec | metric=angular, n_trees=10 | 0.406 | 0.374 |
| Wikipedia | Annoy | Word2Vec | metric=angular, n_trees=100 | 0.540 | 0.316 |
| Wikipedia | Annoy | Word2Vec | metric=angular, n_trees=1000 | 0.541 | 1.667 |
| Wikipedia | Annoy | Word2Vec | metric=euclidean, n_trees=10 | 0.527 | 0.255 |
| Wikipedia | Annoy | Word2Vec | metric=euclidean, n_trees=100 | 0.969 | 0.426 |
| `Wikipedia` | `Annoy` | `Word2Vec` | `metric=euclidean, n_trees=1000` | `1.000` | `0.930` |
| Wikipedia | Annoy | Word2Vec | metric=dot, n_trees=10 | 0.086 | 0.198 |
| Wikipedia | Annoy | Word2Vec | metric=dot, n_trees=100 | 0.058 | 0.247 |
| Wikipedia | Annoy | Word2Vec | metric=dot, n_trees=1000 | 0.053 | 1.329 |


**Run Machine Type:** `macbook pro m1 16gb ram`, but we run in `docker container with ram limit 4gb and 4 cores`

Also, we implemented both algorithms in MinHashLSH(but not optimized version) and Annoy Index( LSH method) and we test it for both possible type of vectores `Tf-idf` and `Word2Vec`.</br></br></br>



2. What happen if we tune the parameters using Airbnb and the run with those parameters in The Wikipedia Dataset? What is the difference in accuracy with the parameters tuned directly in Wikipedia data?

The optimal solution for Wikipedia data is Annoy with features Word2Vec and parameters: `metric=euclidean, n_trees=1000` and accuracy is `1.000`</br>
The optimal solution for Barcelona data is Annoy with features Tf-Idf and parameters: `metric=euclidean, n_trees=1000` and accuracy is `0.919`</br>

If we will use optimal parameters from Barcelona data for Wikipedia data, we will get accuracy `0.910`, which mean that we will lose `0.090` accuracy. It happened because for different data different optimal parameters needed and we cannot used one stack of parameters for all data. 
